In [1]:
from datasets import load_dataset
import seaborn as sns
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader

from transformers import BertForQuestionAnswering, AdamW, BertTokenizer, BertTokenizerFast

/home/chrispy_bacon/miniconda3/envs/bert/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_dataset('squad')
train_dataset, validation_dataset = dataset['train'], dataset['validation']

Reusing dataset squad (/home/chrispy_bacon/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)
100%|██████████| 2/2 [00:00<00:00, 465.03it/s]


In [3]:
def add_end_idx(answers, contexts):
    new_answers = []
    # loop through each answer-context pair
    for answer, context in zip(answers, contexts):
        # quick reformating to remove lists
        answer['text'] = answer['text'][0]
        answer['answer_start'] = answer['answer_start'][0]
        # gold_text refers to the answer we are expecting to find in context
        gold_text = answer['text']
        # we already know the start index
        start_idx = answer['answer_start']
        # and ideally this would be the end index...
        end_idx = start_idx + len(gold_text)

        # ...however, sometimes squad answers are off by a character or two
        if context[start_idx:end_idx] == gold_text:
            # if the answer is not off :)
            answer['answer_end'] = end_idx
        else:
            # this means the answer is off by 1-2 tokens
            for n in [1, 2]:
                if context[start_idx-n:end_idx-n] == gold_text:
                    answer['answer_start'] = start_idx - n
                    answer['answer_end'] = end_idx - n
        new_answers.append(answer)
    return new_answers

def prep_data(dataset):
    questions = dataset['question']
    contexts = dataset['context']
    answers = add_end_idx(
        dataset['answers'],
        contexts
    )
    return {
        'question': questions,
        'context': contexts,
        'answers': answers
    }

def add_token_positions(encodings, answers, tokenizer):
    # initialize lists to contain the token indices of answer start/end
    start_positions = []
    end_positions = []
    # for i in tqdm(range(len(answers))):
    for i in range(len(answers)):
        # append start/end token position using char_to_token method
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        # end position cannot be found, char_to_token found space, so shift position until found
        shift = 1
        while end_positions[-1] is None:
            # encodings = BatchEncoding(answers[i]['answers_end'] -shift, encoding=EncodingFast())
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] - shift)
            shift += 1
    # update our encodings object with the new token-based start/end positions
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [4]:
def get_next_dataloader(start_index: int, end_index: int, shuffle: bool = True, dataset_key: str = 'train', tokenizer=None) -> DataLoader:
    part_dataset = dataset[dataset_key][start_index:end_index]
    part_dataset = prep_data(part_dataset)
    part_train = tokenizer(part_dataset['context'], part_dataset['question'],
                          truncation=True, padding='max_length',
                          max_length=512, return_tensors='pt')
    tokenizer.decode(part_train['input_ids'][0])[:855]
    # apply function to the data
    add_token_positions(part_train, part_dataset['answers'], tokenizer)
    part_dataset = SquadDataset(part_train)
    return DataLoader(part_dataset, batch_size=32, shuffle=shuffle)

In [5]:
train_dataset = prep_data(train_dataset)
validation_dataset = prep_data(validation_dataset)

In [6]:


pretrained_model_name: str = 'bert-large-uncased-whole-word-masking-finetuned-squad'

full_bert = BertForQuestionAnswering.from_pretrained(pretrained_model_name)
full_tokenizer = BertTokenizer.from_pretrained(pretrained_model_name)

Downloading: 100%|██████████| 226k/226k [00:00<00:00, 525kB/s] 
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 23.6kB/s]
Downloading: 100%|██████████| 455k/455k [00:00<00:00, 806kB/s]  


In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [8]:
def get_last_saved_epoch(path: str = 'models/') -> int:
    from os import walk
    files = []
    for (_, _, filenames) in walk(path):
        filenames = [file for file in filenames if file.endswith('.pt')]
        files.extend(filenames)

    epoch_list = [int(file.split('.')[0].split('_').pop()) for file in files]

    return max(epoch_list) if len(epoch_list) > 0 else -1


def init_model(epoch: int = 0):
    model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

    model.to(device)
    
    if epoch != -1:
        model_dict = torch.load(f'models/bert_model_{epoch}.pt', map_location=torch.device('cpu'))
        model.load_state_dict(model_dict)

    return model

In [9]:
saved_epoch = get_last_saved_epoch()
my_model = init_model(saved_epoch)
fast_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [10]:
full_bert.eval()
my_model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [11]:
max_train_batch_length: int = 2738
max_validation_batch_length: int = 331

num_episodes: int = 100
save_zones = [epoch for epoch in range(10, 101, 25)]
step = 5

training_losses = []
validation_losses = []

In [14]:
for start_range in range(0, 1, step):
    end_range = min(max_train_batch_length, start_range + step)

fast_loop = get_next_dataloader(start_range, end_range, False,tokenizer= fast_tokenizer)
full_loop = get_next_dataloader(start_range, end_range, False, tokenizer= full_tokenizer)

ValueError: char_to_token() is not available when using Python based tokenizers